## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [2]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


### 1. Inspect data

In [3]:
# dimensions of experiment

n_total = train_data.shape[0]
print('n_total:', n_total)

n_control = train_data.query('Promotion == "No"').shape[0]
print('n_control:', n_control)

n_experiment = train_data.query('Promotion == "Yes"').shape[0]
print('n_experiment:', n_experiment)

n_total: 84534
n_control: 42170
n_experiment: 42364


In [4]:
# investigate datatypes
train_data.dtypes

ID             int64
Promotion     object
purchase       int64
V1             int64
V2           float64
V3           float64
V4             int64
V5             int64
V6             int64
V7             int64
dtype: object

In [8]:
# total number of purchases

purchases_total = train_data.query('purchase == 1').shape[0]
print('purchases_total:', purchases_total)

purchases_control = train_data.query('Promotion == "No" and purchase == 1').shape[0]
print('purchases_control:', purchases_control)

purchases_experiment = train_data.query('Promotion == "Yes" and purchase == 1').shape[0]
print('purchases_experiment:', purchases_experiment)

purchases_total: 1040
purchases_control: 319
purchases_experiment: 721


In [5]:
# proportion of customers who bought the product

p_total = train_data.purchase.mean()
print('p_total:', p_total)

p_control = train_data.query('Promotion == "No"').purchase.mean()
print('p_control:', p_control)

p_experiment = train_data.query('Promotion == "Yes"').purchase.mean()
print('p_experiment:', p_experiment)

p_total: 0.012302742091939338
p_control: 0.007564619397676073
p_experiment: 0.017019167217448776


### 2. Calculate metrics

In [7]:
# Incremental Response Rate (IRR)

IRR = p_experiment - p_control
print('Incremental Response Rate (IRR):', IRR)

Incremental Response Rate (IRR): 0.009454547819772702


In [9]:
# Net Incremental Revenue (NIR)

NIR = (10 * purchases_experiment - 0.15 * n_experiment) - 10 * purchases_control
print('Net Incremental Revenue (NIR):', NIR)

Net Incremental Revenue (NIR): -2334.5999999999995


### 3. Hypothesis test

1) Build user Funnel --> not relevant here
2) Diversion of Users (event-based. cookie-based, account-based) --> not relevant here
3) Selcting Invariant Metric: Distribution of customers to control group and experiment group.
4) Selecting Evaluation Metric(s): IRR & NIR
5) Use Correction (Bonferroni, Sidark): Bonferroni, because there are two evaluation metrics
6) Experiment size:
7) Validity, Bias, Ehtics
8) Analyze Data

#### Hypothesis regarding the Incremental Response Rate (IRR):
* $H_0: IRR = 0$
* $H_1: IRR > 0$

With a Bonferroni Correction, where $\frac{\alpha}{n_{evaluation metrics}}$
* $\alpha = 2.5\% = 0.025$

Question to answe:
* Is it less probable than alpha = 2.5% to have the observed difference of IRR = 0.009454, assuming there is no difference in Incremental Response between those who receive a a promotion and those who not.

In [3]:
# Invariant Metric

# Check if distribution of amount of observations in each group is probable under the hypothesis that both groups are equally big

#### Hypothesis regarding the INet Incremental Revenue (NIR):
* $H_0: NIR = 0$
* $H_1: NIR > 0$

With a Bonferroni Correction, where $\frac{\alpha}{n_{evaluation metrics}}$
* $\alpha = 2.5\% = 0.025$

In [ ]:
# IRR metric

In [ ]:
# NIR metric

In [ ]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    
    
    
    
    return promotion

In [ ]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)